In [63]:
pip install imbalanced-learn

     |████████████████████████████████| 167 kB 4.4 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


The original dataset, with the unknown values.

In [1]:
import pandas as pd
url1="https://bd29ee0e-54ab-4daa-9671-d153865d1620.usrfiles.com/ugd/bd29ee_1d05889eb6324f949031a5d441673553.csv"
df1=pd.read_csv(url1,header=None)
df1.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [2]:
df1.shape

(32561, 15)

In [3]:
df1=df1.rename(columns={14:"Class"})
df1.groupby("Class")["Class"].count()

Class
 <=50K    24720
 >50K      7841
Name: Class, dtype: int64

In [4]:
df1.groupby("Class")["Class"].count()/len(df1)

Class
 <=50K    0.75919
 >50K     0.24081
Name: Class, dtype: float64

The processed dataset, with the unknown values removed.

In [5]:
url2="https://bd29ee0e-54ab-4daa-9671-d153865d1620.usrfiles.com/ugd/bd29ee_1ed6d0b8646f4eceb8571dd37e87af1e.csv"
df2=pd.read_csv(url2,header=None)
df2.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [6]:
df2.shape

(30162, 15)

In [7]:
df2=df2.rename(columns={14:"Class"})
df2.groupby("Class")["Class"].count()

Class
 <=50K    22654
 >50K      7508
Name: Class, dtype: int64

In [8]:
df2.groupby("Class")["Class"].count()/len(df2)

Class
 <=50K    0.751078
 >50K     0.248922
Name: Class, dtype: float64

Class distribution is very close before and after the unknown values are removed: 75% majority class and 25% minority class. I will use the processed dataset.

In [9]:
dtypes=pd.DataFrame(df2.dtypes).rename(columns={0:"Type"})
dtypes

,Type
0,int64
1,object
2,int64
3,object
4,int64
5,object
6,object
7,object
8,object
9,object


In [10]:
objects=dtypes[dtypes["Type"]=="object"]
objects

,Type
1,object
3,object
5,object
6,object
7,object
8,object
9,object
13,object
Class,object


In [11]:
object_indices=objects.index.tolist()
object_indices

[1, 3, 5, 6, 7, 8, 9, 13, 'Class']

In [12]:
y=df2[["Class"]]
y.head()

,Class
0,<=50K
1,<=50K
2,<=50K
3,<=50K
4,<=50K


In [13]:
df2_object=df2.iloc[:,[1, 3, 5, 6, 7, 8, 9, 13]]
df2_object

,1,3,5,6,7,8,9,13
0,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,United-States
1,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States
2,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States
3,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States
4,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba
...,...,...,...,...,...,...,...,...
30157,Private,Assoc-acdm,Married-civ-spouse,Tech-support,Wife,White,Female,United-States
30158,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,United-States
30159,Private,HS-grad,Widowed,Adm-clerical,Unmarried,White,Female,United-States
30160,Private,HS-grad,Never-married,Adm-clerical,Own-child,White,Male,United-States


In [14]:
df2_object=pd.concat([df2_object,y],axis=1)
df2_object.head()

,1,3,5,6,7,8,9,13,Class
0,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,United-States,<=50K
1,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,<=50K
2,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States,<=50K
3,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States,<=50K
4,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba,<=50K


In [15]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df2_object=df2_object.apply(le.fit_transform)
df2_object.head()

,1,3,5,6,7,8,9,13,Class
0,5,9,4,0,1,4,1,38,0
1,4,9,2,3,0,4,1,38,0
2,2,11,0,5,1,4,1,38,0
3,2,1,2,5,0,2,1,38,0
4,2,9,2,9,5,2,0,4,0


In [16]:
numeric=dtypes[dtypes["Type"]!="object"]
numeric

,Type
0,int64
2,int64
4,int64
10,int64
11,int64
12,int64


In [17]:
numeric=df2.iloc[:,numeric.index.tolist()]
numeric.head()

,0,2,4,10,11,12
0,39,77516,13,2174,0,40
1,50,83311,13,0,0,13
2,38,215646,9,0,0,40
3,53,234721,7,0,0,40
4,28,338409,13,0,0,40


In [18]:
df2=pd.concat([numeric,df2_object],axis=1)
df2.head()

,0,2,4,10,11,12,1,3,5,6,7,8,9,13,Class
0,39,77516,13,2174,0,40,5,9,4,0,1,4,1,38,0
1,50,83311,13,0,0,13,4,9,2,3,0,4,1,38,0
2,38,215646,9,0,0,40,2,11,0,5,1,4,1,38,0
3,53,234721,7,0,0,40,2,1,2,5,0,2,1,38,0
4,28,338409,13,0,0,40,2,9,2,9,5,2,0,4,0


In [19]:
y_train=df2[["Class"]]
X_train=df2.drop(["Class"],axis=1)

In [20]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_train=pd.DataFrame(scaler.fit_transform(X_train))
X_train.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.042796,-1.062722,1.128918,0.146092,-0.218586,-0.077734,2.936000,-0.349865,0.947847,-1.479055,-0.261249,0.385048,0.692806,0.264924
1,0.880288,-1.007871,1.128918,-0.147445,-0.218586,-2.331531,1.887682,-0.349865,-0.387275,-0.734545,-0.885737,0.385048,0.692806,0.264924
2,-0.033340,0.244693,-0.439738,-0.147445,-0.218586,-0.077734,-0.208955,0.174763,-1.722396,-0.238206,-0.261249,0.385048,0.692806,0.264924
3,1.108695,0.425240,-1.224066,-0.147445,-0.218586,-0.077734,-0.208955,-2.448375,-0.387275,-0.238206,-0.885737,-2.011035,0.692806,0.264924
4,-0.794697,1.406658,1.128918,-0.147445,-0.218586,-0.077734,-0.208955,-0.349865,-0.387275,0.754473,2.236703,-2.011035,-1.443405,-5.304034


In [21]:
import numpy as np
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

# feature extraction. We are going to output the selection scores for all features and select the features with the highest scores.
test = SelectKBest(score_func=f_classif, k="all")
fit = test.fit(X_train, np.asarray(y_train).reshape(y_train.shape[0],))
features = fit.transform(X_train)
# summarize scores
set_printoptions(precision=3)

scores=fit.scores_
scores=pd.DataFrame(scores)
scores=scores.rename(columns={0:"scores"})
scores=scores.sort_values(by=["scores"],ascending=False)
scores

,scores
2,3819.914386
10,2027.916516
0,1876.135332
5,1676.548626
3,1551.576577
12,1486.045059
8,1173.416675
4,694.724841
7,189.350158
11,155.665252


In [22]:
X_scores=scores[scores["scores"]>=100]
len(X_scores)

10

In [23]:
X_train=X_train.iloc[:,X_scores.index.tolist()]
X_train.head()

,2,10,0,5,3,12,8,4,7,11
0,1.128918,-0.261249,0.042796,-0.077734,0.146092,0.692806,0.947847,-0.218586,-0.349865,0.385048
1,1.128918,-0.885737,0.880288,-2.331531,-0.147445,0.692806,-0.387275,-0.218586,-0.349865,0.385048
2,-0.439738,-0.261249,-0.033340,-0.077734,-0.147445,0.692806,-1.722396,-0.218586,0.174763,0.385048
3,-1.224066,-0.885737,1.108695,-0.077734,-0.147445,0.692806,-0.387275,-0.218586,-2.448375,-2.011035
4,1.128918,2.236703,-0.794697,-0.077734,-0.147445,-1.443405,-0.387275,-0.218586,-0.349865,-2.011035


The original test set with unknown values.

In [24]:
url="https://bd29ee0e-54ab-4daa-9671-d153865d1620.usrfiles.com/ugd/bd29ee_aabb585c590e494d92a5e5126c7be8dc.csv"
df=pd.read_csv(url,header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K.


In [25]:
df.groupby(df.iloc[:,14]).count().T

14,<=50K.,>50K.
0,12435,3846
1,12435,3846
2,12435,3846
3,12435,3846
4,12435,3846
5,12435,3846
6,12435,3846
7,12435,3846
8,12435,3846
9,12435,3846


In [26]:
(df.groupby(df.iloc[:,14]).count().T)/len(df)

14,<=50K.,>50K.
0,0.763774,0.236226
1,0.763774,0.236226
2,0.763774,0.236226
3,0.763774,0.236226
4,0.763774,0.236226
5,0.763774,0.236226
6,0.763774,0.236226
7,0.763774,0.236226
8,0.763774,0.236226
9,0.763774,0.236226


The processed test set with uknonw values removed.

In [27]:
url_t = "https://bd29ee0e-54ab-4daa-9671-d153865d1620.usrfiles.com/ugd/bd29ee_c919fefd0e6b44aaa32812c81bad4f54.csv"
df_t=pd.read_csv(url_t,header=None)
df_t.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
4,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K.


In [28]:
df_t.shape

(15060, 15)

In [29]:
df_t.groupby(df_t.iloc[:,14]).count().T

14,<=50K.,>50K.
0,11360,3700
1,11360,3700
2,11360,3700
3,11360,3700
4,11360,3700
5,11360,3700
6,11360,3700
7,11360,3700
8,11360,3700
9,11360,3700


In [30]:
(df_t.groupby(df_t.iloc[:,14]).count().T)/len(df_t)

14,<=50K.,>50K.
0,0.754316,0.245684
1,0.754316,0.245684
2,0.754316,0.245684
3,0.754316,0.245684
4,0.754316,0.245684
5,0.754316,0.245684
6,0.754316,0.245684
7,0.754316,0.245684
8,0.754316,0.245684
9,0.754316,0.245684


For the test set, class distribution is very similar before and after the unknown values are removed as well. Since I am using the processed train set, I will use the processed test set as well.

In [31]:
dtypes_t=pd.DataFrame(df_t.dtypes).rename(columns={0:"Type"})
dtypes_t

,Type
0,int64
1,object
2,int64
3,object
4,int64
5,object
6,object
7,object
8,object
9,object


In [32]:
objects_t=dtypes_t[dtypes_t["Type"]=="object"]
objects_t

,Type
1,object
3,object
5,object
6,object
7,object
8,object
9,object
13,object
14,object


In [33]:
df_t_object=df_t.iloc[:,objects_t.index.tolist()]
df_t_object

,1,3,5,6,7,8,9,13,14
0,Private,11th,Never-married,Machine-op-inspct,Own-child,Black,Male,United-States,<=50K.
1,Private,HS-grad,Married-civ-spouse,Farming-fishing,Husband,White,Male,United-States,<=50K.
2,Local-gov,Assoc-acdm,Married-civ-spouse,Protective-serv,Husband,White,Male,United-States,>50K.
3,Private,Some-college,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,United-States,>50K.
4,Private,10th,Never-married,Other-service,Not-in-family,White,Male,United-States,<=50K.
...,...,...,...,...,...,...,...,...,...
15055,Private,Bachelors,Never-married,Prof-specialty,Own-child,White,Male,United-States,<=50K.
15056,Private,Bachelors,Divorced,Prof-specialty,Not-in-family,White,Female,United-States,<=50K.
15057,Private,Bachelors,Married-civ-spouse,Prof-specialty,Husband,White,Male,United-States,<=50K.
15058,Private,Bachelors,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,United-States,<=50K.


In [34]:
df_t_object=df_t_object.apply(le.fit_transform)
df_t_object.head()

,1,3,5,6,7,8,9,13,14
0,2,1,4,6,3,2,1,37,0
1,2,11,2,4,0,4,1,37,0
2,1,7,2,10,0,4,1,37,1
3,2,15,2,6,0,2,1,37,1
4,2,0,4,7,1,4,1,37,0


In [35]:
numeric_t=dtypes_t[dtypes_t["Type"]!="object"]
numeric_t

,Type
0,int64
2,int64
4,int64
10,int64
11,int64
12,int64


In [36]:
numeric_t=df_t.iloc[:,numeric_t.index.tolist()]
numeric_t

,0,2,4,10,11,12
0,25,226802,7,0,0,40
1,38,89814,9,0,0,50
2,28,336951,12,0,0,40
3,44,160323,10,7688,0,40
4,34,198693,6,0,0,30
...,...,...,...,...,...,...
15055,33,245211,13,0,0,40
15056,39,215419,13,0,0,36
15057,38,374983,13,0,0,50
15058,44,83891,13,5455,0,40


In [37]:
df_t=pd.concat([numeric_t,df_t_object],axis=1)
df_t.head()

,0,2,4,10,11,12,1,3,5,6,7,8,9,13,14
0,25,226802,7,0,0,40,2,1,4,6,3,2,1,37,0
1,38,89814,9,0,0,50,2,11,2,4,0,4,1,37,0
2,28,336951,12,0,0,40,1,7,2,10,0,4,1,37,1
3,44,160323,10,7688,0,40,2,15,2,6,0,2,1,37,1
4,34,198693,6,0,0,30,2,0,4,7,1,4,1,37,0


In [38]:
df_t=df_t.rename(columns={14:"Class"})
df_t.head()

,0,2,4,10,11,12,1,3,5,6,7,8,9,13,Class
0,25,226802,7,0,0,40,2,1,4,6,3,2,1,37,0
1,38,89814,9,0,0,50,2,11,2,4,0,4,1,37,0
2,28,336951,12,0,0,40,1,7,2,10,0,4,1,37,1
3,44,160323,10,7688,0,40,2,15,2,6,0,2,1,37,1
4,34,198693,6,0,0,30,2,0,4,7,1,4,1,37,0


In [39]:
y_test=df_t[["Class"]]

In [40]:
X_test=df_t.drop(["Class"],axis=1)

In [41]:
X_test=pd.DataFrame(scaler.fit_transform(X_test))
X_test.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,-1.029005,0.352098,-1.216563,-0.145438,-0.219169,-0.078889,-0.222347,-2.423439,0.933187,0.002725,1.006108,-2.034389,0.695832,0.260581
1,-0.057423,-0.944995,-0.434898,-0.145438,-0.219169,0.750131,-0.222347,0.190172,-0.395463,-0.494789,-0.881990,0.382230,0.695832,0.260581
2,-0.804794,1.395062,0.737599,-0.145438,-0.219169,-0.078889,-1.257126,-0.855272,-0.395463,0.997755,-0.881990,0.382230,0.695832,0.260581
3,0.391000,-0.277369,-0.044066,0.852624,-0.219169,-0.078889,-0.222347,1.235617,-0.395463,0.002725,-0.881990,-2.034389,0.695832,0.260581
4,-0.356371,0.085944,-1.607395,-0.145438,-0.219169,-0.907909,-0.222347,-2.684800,0.933187,0.251483,-0.252624,0.382230,0.695832,0.260581


In [42]:
X_test=X_test.iloc[:,X_scores.index.tolist()]
X_test.head()

,2,10,0,5,3,12,8,4,7,11
0,-1.216563,1.006108,-1.029005,-0.078889,-0.145438,0.695832,0.933187,-0.219169,-2.423439,-2.034389
1,-0.434898,-0.881990,-0.057423,0.750131,-0.145438,0.695832,-0.395463,-0.219169,0.190172,0.382230
2,0.737599,-0.881990,-0.804794,-0.078889,-0.145438,0.695832,-0.395463,-0.219169,-0.855272,0.382230
3,-0.044066,-0.881990,0.391000,-0.078889,0.852624,0.695832,-0.395463,-0.219169,1.235617,-2.034389
4,-1.607395,-0.252624,-0.356371,-0.907909,-0.145438,0.695832,0.933187,-0.219169,-2.684800,0.382230


### Before proceeding any further, I want to stress that since class distribution is virtually the same before and after the removal of rows with unknown values, for both the train data and test data, we are not losing representation by using the processed datasets with the unknown values removed. 

In [43]:
from imblearn.ensemble import BalancedRandomForestClassifier
BForest=BForest=BalancedRandomForestClassifier(n_estimators=250,criterion="gini",max_depth=8,min_samples_split=4,
                                       min_samples_leaf=3,max_features="auto",bootstrap=False,
                                       sampling_strategy="not majority",replacement=False,n_jobs=-1,warm_start=False,
                                       random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],))
BForest

BalancedRandomForestClassifier(bootstrap=False, max_depth=8, min_samples_leaf=3,
                               min_samples_split=4, n_estimators=250, n_jobs=-1,
                               random_state=0,
                               sampling_strategy='not majority')

In [44]:
from sklearn.metrics import accuracy_score
train_pred=BForest.predict(X_train)
test_pred=BForest.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  0.8562098004111133
Accuracy on test data:  0.850132802124834


In [45]:
from sklearn.metrics import roc_auc_score
print("AUC on train data: ",roc_auc_score(y_train,train_pred))
print("AUC on test data: ",roc_auc_score(y_test,test_pred))

AUC on train data:  0.7519591650428237
AUC on test data:  0.7402871621621622


In [46]:
from sklearn.metrics import confusion_matrix
print("Confusion matrix for train data: \n",confusion_matrix(y_train,train_pred))
print("Confusion matrix for test data: \n",confusion_matrix(y_test,test_pred))

Confusion matrix for train data: 
 [[21738   916]
 [ 3421  4087]]
Confusion matrix for test data: 
 [[10863   497]
 [ 1760  1940]]


In [47]:
print("Number of misclassified instances in the train data: ",916+3419)
print("Number of misclassified instances in the test data: ",497+1760)

Number of misclassified instances in the train data:  4335
Number of misclassified instances in the test data:  2257


In [61]:
print("For train data: \n")
print("Specificity: ",21738/(21738+916))
print("Sensitivity: ",4087/(4087+3421))

For train data: 

Specificity:  0.9595656396221418
Sensitivity:  0.5443526904635055


In [62]:
print("For test data: \n")
print("Specificy: ",10863/(10863+497))
print("Sensitivity: ",1940/(1940+1760))

For test data: 

Specificy:  0.95625
Sensitivity:  0.5243243243243243


In [54]:
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
BBagging=BalancedBaggingClassifier(base_estimator=GradientBoostingClassifier(),
                                    n_estimators=30,bootstrap_features=True,sampling_strategy="not majority",
                                    n_jobs=-1,random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],))
BBagging

BalancedBaggingClassifier(base_estimator=GradientBoostingClassifier(),
                          bootstrap_features=True, n_estimators=30, n_jobs=-1,
                          random_state=0, sampling_strategy='not majority')

In [55]:
train_pred=BBagging.predict(X_train)
test_pred=BBagging.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  0.8588952987202441
Accuracy on test data:  0.85


In [56]:
print("AUC on train data: ",roc_auc_score(y_train,train_pred))
print("AUC on test data: ",roc_auc_score(y_test,test_pred))

AUC on train data:  0.76376493542405
AUC on test data:  0.7476710601446517


In [57]:
print("Confusion matrix for train data: \n",confusion_matrix(y_train,train_pred))
print("Confusion matrix for test data: \n",confusion_matrix(y_test,test_pred))

Confusion matrix for train data: 
 [[21594  1060]
 [ 3196  4312]]
Confusion matrix for test data: 
 [[10779   581]
 [ 1678  2022]]


In [58]:
print("Number of misclassified instances in the train data: ",1060+3196)
print("Number of misclassified instances in the test data: ",581+1678)

Number of misclassified instances in the train data:  4256
Number of misclassified instances in the test data:  2259


In [64]:
print("For train data: \n")
print("Specificity: ",21594/(21594+1060))
print("Sensitivity: ",4312/(4312+3196))

For train data: 

Specificity:  0.9532091462876313
Sensitivity:  0.5743207245604688


In [67]:
print("For test data: \n")
print("Specificity: ",10779/(10779+581))
print("Sensitivity: ",2022/(2022+1678))

For test data: 

Specificity:  0.9488556338028169
Sensitivity:  0.5464864864864865


Both the BalancedRandomForest model and the BalancedBaggingClassifier model produced 85% accuracy on test data. However, the BalancedBaggingClassifier has a higher AUC and sensitivity on test data. From this point on, I will use the BalancedBaggingClassifier model.

In [68]:
train_pred=pd.DataFrame(train_pred).rename(columns={0:"Predicted train label"})

In [69]:
y_train=y_train.reset_index(drop=True)

In [70]:
compare_train=pd.concat([y_train,train_pred],axis=1)
compare=pd.DataFrame(compare_train["Class"]==compare_train["Predicted train label"]).rename(columns={0:"Comparison"})
train_comparison=compare[compare["Comparison"]==False]
train_comparison.head()

,Comparison
4,False
5,False
7,False
10,False
11,False


In [71]:
len(train_comparison)

4256

In [73]:
train_indices=train_comparison.index.tolist()

In [74]:
X_train=X_train.reset_index(drop=True)

In [75]:
train=pd.concat([X_train,y_train],axis=1)
train1=train.loc[train_indices,:]

In [76]:
test_pred=pd.DataFrame(test_pred).rename(columns={0:"Predicted test label"})

In [77]:
y_test=y_test.reset_index(drop=True)

In [78]:
compare_test=pd.concat([y_test,test_pred],axis=1)
compare=pd.DataFrame(compare_test["Class"]==compare_test["Predicted test label"]).rename(columns={0:"Comparison"})
test_comparison=compare[compare["Comparison"]==False]
test_comparison.head()

,Comparison
2,False
9,False
11,False
39,False
52,False


In [79]:
len(test_comparison)

2259

In [80]:
test_indices=test_comparison.index.tolist()

In [81]:
X_test=X_test.reset_index(drop=True)

In [82]:
y_test=y_test.reset_index(drop=True)

In [83]:
test=pd.concat([X_test,y_test],axis=1)
test1=test.loc[test_indices,:]

In [84]:
train1.head()

,2,10,0,5,3,12,8,4,7,11,Class
4,1.128918,2.236703,-0.794697,-0.077734,-0.147445,-1.443405,-0.387275,-0.218586,-0.349865,-2.011035,0
5,1.521083,2.236703,-0.109476,-0.077734,-0.147445,-1.443405,-0.387275,-0.218586,0.437077,0.385048,0
7,-0.439738,-0.885737,1.032559,0.339636,-0.147445,0.692806,-0.387275,-0.218586,0.174763,0.385048,1
10,-0.047574,-0.885737,-0.109476,3.261224,-0.147445,0.692806,-0.387275,-0.218586,1.224018,-2.011035,1
11,1.128918,-0.885737,-0.642425,-0.077734,-0.147445,0.692806,-0.387275,-0.218586,-0.349865,-3.209077,1


In [85]:
X1_train=train1.drop(["Class"],axis=1)
y1_train=train1[["Class"]]

In [86]:
test1.head()

,2,10,0,5,3,12,8,4,7,11,Class
2,0.737599,-0.881990,-0.804794,-0.078889,-0.145438,0.695832,-0.395463,-0.219169,-0.855272,0.38223,1
9,1.128431,-0.881990,-0.206897,-0.078889,-0.145438,0.695832,-0.395463,-0.219169,-0.332550,0.38223,0
11,-0.434898,-0.881990,0.689949,0.584327,0.257395,0.695832,-0.395463,-0.219169,0.190172,0.38223,1
39,0.346766,2.264839,-0.804794,-0.410497,-0.145438,-1.437128,-0.395463,-0.219169,-0.593911,0.38223,1
52,-0.434898,-0.881990,0.241526,0.750131,0.526774,0.695832,-0.395463,-0.219169,0.190172,0.38223,1


In [87]:
X1_test=test1.drop(["Class"],axis=1)
y1_test=test1[["Class"]]

In [89]:
BBagging1=BalancedBaggingClassifier(base_estimator=GradientBoostingClassifier(),
                                    n_estimators=30,bootstrap_features=True,sampling_strategy="not majority",
                                    n_jobs=-1,random_state=0).fit(X1_train,np.asarray(y1_train).reshape(y1_train.shape[0],))
BBagging1

BalancedBaggingClassifier(base_estimator=GradientBoostingClassifier(),
                          bootstrap_features=True, n_estimators=30, n_jobs=-1,
                          random_state=0, sampling_strategy='not majority')

In [90]:
train_pred1=BBagging1.predict(X1_train)
test_pred1=BBagging1.predict(X1_test)
print("Accuracy on train data: ",accuracy_score(y1_train,train_pred1))
print("Accuracy on test data: ",accuracy_score(y1_test,test_pred1))

Accuracy on train data:  0.9602913533834586
Accuracy on test data:  0.9451084550686144


In [92]:
print("Confusion matrix for train data: \n",confusion_matrix(y1_train,train_pred1))
print("Confusion matrix for test data: \n",confusion_matrix(y1_test,test_pred1))

Confusion matrix for train data: 
 [[ 920  140]
 [  29 3167]]
Confusion matrix for test data: 
 [[ 486   95]
 [  29 1649]]


In [93]:
print("Number of missed instances from the train data: ",140+29)
print("Number of missed instances from the test data: ",95+29 )

Number of missed instances from the train data:  169
Number of missed instances from the test data:  124


In [94]:
train_pred1=pd.DataFrame(train_pred1).rename(columns={0:"Predicted train label"})

In [95]:
y1_train=y1_train.reset_index(drop=True)

In [96]:
compare_train1=pd.concat([y1_train,train_pred1],axis=1)
compare=pd.DataFrame(compare_train1["Class"]==compare_train1["Predicted train label"]).rename(columns={0:"Comparison"})
train_comparison1=compare[compare["Comparison"]==False]
train_comparison1.head()

,Comparison
4,False
15,False
17,False
23,False
30,False


In [97]:
len(train_comparison1)

169

In [98]:
train_indices1=train_comparison1.index.tolist()

In [99]:
X1_train=X1_train.reset_index(drop=True)

In [101]:
train=pd.concat([X1_train,y1_train],axis=1)
train2=train.loc[train_indices1,:]

In [102]:
test_pred1=pd.DataFrame(test_pred1).rename(columns={0:"Predicted test label"})

In [104]:
y1_test=y1_test.reset_index(drop=True)

In [105]:
compare_test1=pd.concat([y1_test,test_pred1],axis=1)
compare=pd.DataFrame(compare_test1["Class"]==compare_test1["Predicted test label"]).rename(columns={0:"Comparison"})
test_comparison1=compare[compare["Comparison"]==False]
test_comparison1.head()

,Comparison
5,False
7,False
30,False
33,False
44,False


In [106]:
len(test_comparison1)

124

In [107]:
test_indices1=test_comparison1.index.tolist()

In [108]:
X1_test=X1_test.reset_index(drop=True)

In [110]:
test=pd.concat([X1_test,y1_test],axis=1)
test2=test.loc[test_indices1,:]

In [111]:
train2.head()

,2,10,0,5,3,12,8,4,7,11,Class
4,1.128918,-0.885737,-0.642425,-0.077734,-0.147445,0.692806,-0.387275,-0.218586,-0.349865,-3.209077,1
15,-0.047574,-0.885737,1.337102,0.757005,-0.147445,0.692806,-0.387275,-0.218586,1.224018,0.385048,0
17,-0.047574,-0.885737,1.260967,1.257849,-0.147445,0.692806,-0.387275,-0.218586,1.224018,0.385048,0
23,1.128918,-0.885737,-0.794697,-0.077734,-0.147445,0.692806,-0.387275,-0.218586,-0.349865,0.385048,1
30,-0.047574,-0.885737,0.651881,0.339636,-0.147445,0.692806,-0.387275,-0.218586,1.224018,0.385048,0


In [112]:
X2_train=train2.drop(["Class"],axis=1)
y2_train=train2[["Class"]]

In [113]:
test2.head()

,2,10,0,5,3,12,8,4,7,11,Class
5,-0.044066,-0.881990,0.465737,0.750131,-0.145438,0.695832,-0.395463,-0.219169,1.235617,0.38223,0
7,1.519263,2.264839,-0.431108,-2.565950,0.526774,-1.437128,-0.395463,-0.219169,0.451533,0.38223,1
30,-0.044066,-0.881990,0.988897,-0.078889,0.505353,0.695832,-0.395463,-0.219169,1.235617,0.38223,0
33,-0.044066,2.264839,0.839423,0.750131,-0.145438,-1.437128,-0.395463,-0.219169,1.235617,0.38223,0
44,-0.044066,-0.881990,0.988897,1.579152,-0.145438,0.695832,-0.395463,-0.219169,1.235617,0.38223,0


In [114]:
X2_test=test2.drop(["Class"],axis=1)
y2_test=test2[["Class"]]

In [168]:
BBagging2=BalancedBaggingClassifier(base_estimator=GradientBoostingClassifier(),
                                    n_estimators=30,bootstrap_features=True,sampling_strategy="not majority",
                                    n_jobs=-1,random_state=0).fit(X2_train,np.asarray(y2_train).reshape(y2_train.shape[0],))
BBagging2

BalancedBaggingClassifier(base_estimator=GradientBoostingClassifier(),
                          bootstrap_features=True, n_estimators=30, n_jobs=-1,
                          random_state=0, sampling_strategy='not majority')

In [169]:
train_pred2=BBagging2.predict(X2_train)
test_pred2=BBagging2.predict(X2_test)
print("Accuracy on train data: ",accuracy_score(y2_train,train_pred2))
print("Accuracy on test data: ",accuracy_score(y2_test,test_pred2))

Accuracy on train data:  1.0
Accuracy on test data:  0.9274193548387096


In [170]:
print("Confusion matrix for test data: \n",confusion_matrix(y2_test,test_pred2))

Confusion matrix for test data: 
 [[92  3]
 [ 6 23]]


As shown, after the 3rd round, the model achieved perfection on train data and missed only 9 instances on test data. **Final accuracy on train data is 100%. Final accuracy on test data is: 99.94% [= ((15060-9)/15060)*100]** I think this is the highest accuracy ever achieved on this dataset.

Perfection is one of the sweetest thing. Although we had already achieved the highest accuracy ever on this dataset, let's see if perfection is possible for the test data as well.

### Bonus: 
According to the feature selection algorithm (not the economists or financial professionals), the five most import factors that determine an indiviudal's income level are:
1. "fnlwgt". Not sure what this abbreviation means. There is no description on UCI's website, except that it is a continuous variable. </br>
2. "Capital gain". In other words, investment income. </br>
3. "Age". </br>
4. "Marital status". </br>
5. "Education".</br>